In [ ]:
## 2 ways to improve the performance
# yolov2 : changes in model configurations
# yolo9000 : changes in model training method. joint training of classification and object detection.


In [1]:
import numpy as np
import matplotlib.pyplot as plt

from global_utils import IoU

## Anchor Boxes

### k-mean clustering

In [ ]:
## https://towardsdatascience.com/create-your-own-k-means-clustering-algorithm-in-python-d7d4c9077670

def KmeanClustering(bboxes, max_iters, k = 5) :
    '''
    read out whole bounding boxes from train datasets, and perform k mean clusterings
    bboxes = (x_center, y_center, w, h, c). Here ratios are respect to the image size, NOT grid cell.
    max_iters : number of maximum iterations
    k : number of clusters
    '''

    w = bboxes[:, 2]
    h = bboxes[:, 3]

    # initialize the centroid
    min_w, max_w = min(w), max(w)
    min_h, max_h = min(h), max(h)
    # uniformly distribute the k cluster centroids
    centroids = [((max_w - min_w) * (i+1), (max_h - min_h) * (i+1)) for i in range(k)]

    prev_centroids = None
    iteration = 0


    while np.not_equal(centroids, prev_centroid).any() or iteration < max_iters :
        
        sorted_points = [[] for _ in range(k)]

        for (_, _, w, h, _) in bboxes :
            
            # Suppose x_center and y_center are same for centroid and data point here, we just want the ratio of w, h.
            x, y = 0.5, 0.5



            dist = np.array([IoU(np.array([x,y,w,h]), np.array([x,y,centroid_w, centroid_h]) ) \
                                for (centroid_w, centroid_h) in centroids])

            centroid_idx = np.argmax(dist)
            sorted_points[centroid_idx].append([w, h])

        prev_centroids = centroids
        sorted_points = np.array(sorted_points)
        centroids = [(np.mean(cluster[:,0]), np.mean(cluster[:,1]) ) for cluster in sorted_points]

        # if any new centroids has NaN, then substitute back to previous centroids
        centroids[np.isnan(centroids)] = prev_centroids[np.isnan(centroids)]

        iteration += 1

    return centroids



In [6]:
samp = np.array([[np.NaN, 0, 0, 0]])
prev = np.array([[1, 2, 3, 4]])
samp[np.isnan(samp)] = prev[np.isnan(samp)]


samp

array([[1., 0., 0., 0.]])

In [ ]:
from global_utils import IoU

## MAP
def calculate_PR(pred, gt, iou_threshold, confidence_threshold) :
    '''
    calculate precision and recall per image per classes
    Recall = TP / (TP+FN)
    Precision = TP / (TP + FP)
    1. match preds and gts using IoU
    3. matched preds will be TP, and remaining unmatched preds will be FP, and unmatched gt are FN.
    '''

    # bboxes = [[x,y,w,h,c],...]. c = confidence_score. filter predicted bboxes by confidence_threshold.
    pred = np.array([p for p in pred if p[-1] > confidence_threshold])
    
    # 행 : pred, 열 : gt
    iou_table = np.zeros((len(pred), len(gt)))
    for j, pred_bbox in enumerate(pred) :
        for i, gt_bbox in enumerate(gt) :
            iou_table[j][i] = IoU(pred_bbox, gt_bbox)
            
    result = iou_table > iou_threshold

    TP = result.any(axis = 1).sum()
    FP = len(pred) - TP
    FN = len(gt) - result.any(axis = 0).sum()
    
    if TP == 0 :
        precision = 0
        recall = 0
    else :
        precision = TP / (TP + FP + 1e-9)
        recall = TP / (TP + FN + 1e-9)

    return precision , recall

    


def MeanAveragePrecision(preds, gts, confidence_threshold, iou_threshold, classes,) :

    

In [15]:
samp = np.array([[True, False, True], 
                 [False,False, True]])

samp.any(axis = 1)

array([ True,  True])

In [11]:
samp.all(axis = 1)

array([ True, False])